# Word Sense Disambiguation (снятие лексической неоднозначности) и Word Sense Induction (нахождение значений слова)
## Arshinov Girgory

In [6]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

from typing import Dict

class FasterMorphology:
    
    def __init__(self):
        self.__morph = MorphAnalyzer()
        self.__cache = {}
    
    def parse(self, word: str):
        if word in self.__cache:
            return self.__cache[word]
        else:
            analysis = self.__morph.parse(word)
            self.__cache[word] = analysis[0]
            return analysis[0]


morph = FasterMorphology()

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word).normal_form for word in words if word]

    return words

In [2]:
# модель обученная на большом корпусе (острожно 1.5 гб)
# !curl 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib' --output all.a010.p10.d300.w5.m100.nonorm.slim.joblib
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

## Задание № 1. Протестировать адаграм в определении перефразирования

In [3]:
import pandas as pd

In [11]:
DATA_ROOT = Path('../compling_nlp_hse_course/data')

In [12]:
DATA_ROOT.joinpath('paraphraser/paraphrases.xml').exists()

True

In [13]:
corpus_xml = html.fromstring(DATA_ROOT.joinpath('paraphraser/paraphrases.xml').open('rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [14]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

Векторизуйте пары текстов с помощью Адаграма, обучите любую модель и оцените качество (кросс-валидацией). 

За основу возьмите код из предыдущего семинара/домашки, только в функции get_embedding вам нужно выбирать вектор нужного значения (импользуйте model.disambiguate и model.sense_vector). Отдельные векторы усредните как и в предыдущем семинаре.

Для вытаскивания пар (целевое слово, контекстые слова) вам нужно будет написать специальную функцию.

In [15]:
# проверяте на списке из чисел, чтобы было удобно дебажить
words = list(range(0, 10))

def get_words_in_context(words, window=3):
    words_in_context = []
    for idx, word in enumerate(words):
        context = []
        for ctx_idx, ctx_word in enumerate(words):
            residual = abs(idx - ctx_idx)
            if residual > 0 and residual <= window:
                context.append(ctx_word)
        words_in_context.append([word, context])
    return words_in_context

In [16]:
test_output = [[0, [1, 2, 3]],
 [1, [0, 2, 3, 4]],
 [2, [0, 1, 3, 4, 5]],
 [3, [0, 1, 2, 4, 5, 6]],
 [4, [1, 2, 3, 5, 6, 7]],
 [5, [2, 3, 4, 6, 7, 8]],
 [6, [3, 4, 5, 7, 8, 9]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

In [17]:
assert get_words_in_context(words) == test_output

Когда получиться такой же результат, добавьте эту функцию в get_embedding. Проходите циклом по результату работы get_words_in_context и поставляйте каждый элемент-список в model.disambiguate.

In [18]:
from typing import List

def get_embedding_adagram(text: List[str], model: adagram.model.VectorModel, window: int = 3):
    
    word2context = get_words_in_context(text, window)
    
    
    vectors = np.zeros((len(word2context), model.dim))
    
    for i, (word, context) in enumerate(word2context):
        
        try:
            sense_id = model.disambiguate(word, context).argmax()
            vectors[i] = model.sense_vector(word, sense_id)
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [19]:
dim = vm.dim
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding_adagram(text, vm)

for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding_adagram(text, vm)

In [20]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [21]:
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report

In [22]:
y = data['label'].values
print(y.shape)
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

(7227,)
              precision    recall  f1-score   support

          -1       0.55      0.51      0.53       629
           0       0.47      0.53      0.50       737
           1       0.36      0.33      0.35       441

    accuracy                           0.47      1807
   macro avg       0.46      0.46      0.46      1807
weighted avg       0.47      0.47      0.47      1807



In [23]:
pd.DataFrame(cross_validate(clf, X_text_w2v, y, cv=4, n_jobs=4))

,fit_time,score_time,test_score
0,5.142710,0.037201,0.415053
1,5.178780,0.032576,0.471500
2,5.280135,0.029135,0.412286
3,5.254598,0.028071,0.394795


## WordNet

Выводить значения просто из текста тяжело, поэтому можно попробовать воспользоваться словарями (wsi сделанное людьми). Для русского придется парсить сайты словарей, а для английского можно воспользоваться WordNet'ом (https://wordnet.princeton.edu/), который есть в nltk.

WordNet - это лексическая база данных, где существительные, прилагательные и глаголы английского сгруппированы по значению. К тому же между ними установлены связи (гипонимия, гипоронимия, миронимия и т.п.).

In [24]:
# запустите если не установлен ворднет
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/grigoriyarshinov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

И даже примеры:

### Задание 2. Реализовать алгоритм Леска и проверить его на реальном датасете

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [25]:
def lesk(word, sentence):
    """Ваш код тут"""
    sentence = set(sentence)
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)
    for i, syns in enumerate(synsets):
        definition = set(syns.definition().lower().split())
        overlap = len(sentence & definition)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    return bestsense

In [26]:
lesk('day', 'some point or period in time'.split())

1

Работать функция должна как-то так:

In [88]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time'.split()) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

**Проверьте насколько хорошо работает такой метод на реальном датасете.** http://lcl.uniroma1.it/wsdeval/ - большой фреймворк для оценки WSD. Там много данных и я взял кусочек, чтобы не было проблем с памятью

In [27]:
from bs4 import BeautifulSoup

In [117]:
!ls data/WSD_Unified_Evaluation_Datasets/ALL/

ALL.data.xml     ALL.gold.key.txt


In [28]:
with open('data/WSD_Unified_Evaluation_Datasets/ALL/ALL.data.xml') as file:
    soup = BeautifulSoup(file.read(), features='lxml')
with open('data/WSD_Unified_Evaluation_Datasets/ALL/ALL.gold.key.txt') as file:
    sense_keys = {line.split()[0]: line.split()[1] for line in file}

In [29]:
from bs4 import NavigableString

In [30]:
texts = []

not_navigable_string = lambda x: not isinstance(x, NavigableString)

for text in filter(not_navigable_string, soup.corpus):
    text_sentences = []
    for sentence in filter(not_navigable_string, text):
        sentence_words = []
        for word in filter(not_navigable_string, sentence):
            sentence_words.append({'word': word.text.strip(),
                             'sense_id': word.get('id'),
                             'lemma': word.get('lemma')})
        text_sentences.append(sentence_words)
    texts.append(text_sentences)

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [31]:
from copy import copy

In [32]:
match_count = 0
word_count = 0
for text in texts[0:1]:
    for sentence in text:
        sentence_tokens = [word["word"] for word in sentence]
        for idx, word in enumerate(sentence):
            sense_id = word.get("sense_id")
            context = copy(sentence_tokens)
            del context[idx]
            if sense_id:
                word_count += 1
                actual_synset = wn.lemma_from_key(sense_keys[sense_id]).synset()
                predicted_synset = wn.synsets(word['lemma'])[lesk(word['lemma'], context)]
                if predicted_synset == actual_synset:
                    match_count += 1

print(match_count / word_count)
        
                    

0.3080495356037152


**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

### Дополнительный балл

Если хотите заработать дополнительный балл, попробуйте улучшить алгоритм Леска любым способом (например, использовать расстояние редактирования вместо пересечения или даже вставить машинное обучение)